In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00


## Import Libraries

In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset
from transformers.pipelines.pt_utils import KeyDataset
import os
import torch
import pandas as pd

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0"  if torch.cuda.is_available() else "cpu")

## Load Emotion Model

In [5]:
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emotion-multilabel-latest", return_all_scores=True,device=device )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [6]:
pipe("Stop w ’we have to worry about the children’ No we do not-many R >20yrs old Go home and make your country better or enter ours legally we can’t afford them#NODACA")

[[{'label': 'anger', 'score': 0.28689053654670715},
  {'label': 'anticipation', 'score': 0.2280251383781433},
  {'label': 'disgust', 'score': 0.42105063796043396},
  {'label': 'fear', 'score': 0.26769909262657166},
  {'label': 'joy', 'score': 0.013531041331589222},
  {'label': 'love', 'score': 0.0023378203622996807},
  {'label': 'optimism', 'score': 0.2784470021724701},
  {'label': 'pessimism', 'score': 0.2713361084461212},
  {'label': 'sadness', 'score': 0.35492101311683655},
  {'label': 'surprise', 'score': 0.010908824391663074},
  {'label': 'trust', 'score': 0.023149754852056503}]]

In [7]:
[[{'label': 'anger', 'score': 0.133321613073349},
  {'label': 'anticipation', 'score': 0.2689744830131531},
  {'label': 'disgust', 'score': 0.24493607878684998},
  {'label': 'fear', 'score': 0.2876245975494385},
  {'label': 'joy', 'score': 0.018130697309970856},
  {'label': 'love', 'score': 0.002839383902028203},
  {'label': 'optimism', 'score': 0.3733889162540436},
  {'label': 'pessimism', 'score': 0.2678101062774658},
  {'label': 'sadness', 'score': 0.3561314642429352},
  {'label': 'surprise', 'score': 0.00995288323611021},
  {'label': 'trust', 'score': 0.02865920588374138}]]

[[{'label': 'anger', 'score': 0.133321613073349},
  {'label': 'anticipation', 'score': 0.2689744830131531},
  {'label': 'disgust', 'score': 0.24493607878684998},
  {'label': 'fear', 'score': 0.2876245975494385},
  {'label': 'joy', 'score': 0.018130697309970856},
  {'label': 'love', 'score': 0.002839383902028203},
  {'label': 'optimism', 'score': 0.3733889162540436},
  {'label': 'pessimism', 'score': 0.2678101062774658},
  {'label': 'sadness', 'score': 0.3561314642429352},
  {'label': 'surprise', 'score': 0.00995288323611021},
  {'label': 'trust', 'score': 0.02865920588374138}]]

## Load Dataset

In [8]:
# Load dataset from Hugging Face
dataset = load_dataset("Thushalya/Davidson_Random_Sampled")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Get Train,Dev,Test Splits

In [9]:
train_dataset=dataset['train']

In [10]:
test_dataset=dataset['test']

In [11]:
# Function to get predictions and store them in the dataset as vectors
def get_predictions_and_store(dataset):
    prediction = pipe(dataset['text'])
    df = [pd.DataFrame(d).set_index('label').transpose() for d in prediction]
    result_df = pd.concat(df, ignore_index=True)
    result_df = result_df.reset_index(drop=True)
    dataset_with_predictions=dataset.add_column("anger",result_df['anger'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("anticipation",result_df['anticipation'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("disgust",result_df['disgust'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("fear",result_df['fear'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("joy",result_df['joy'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("love",result_df['love'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("optimism",result_df['optimism'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("pessimism",result_df['pessimism'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("sadness",result_df['sadness'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("surprise",result_df['surprise'].tolist())
    dataset_with_predictions=dataset_with_predictions.add_column("trust",result_df['trust'].tolist())

    return  dataset_with_predictions

## Get predictions and update the dataset

In [12]:
train_dataset_with_predictions = get_predictions_and_store(train_dataset)

In [13]:
test_dataset_with_predictions = get_predictions_and_store(test_dataset)

## Save Datasets as CSV Files

In [16]:
train_dataset_with_predictions.to_csv('/content/drive/MyDrive/Krishnin/Davidson_sampled_with_emotion_train.csv')

Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

1751564

In [17]:
test_dataset_with_predictions.to_csv('/content/drive/MyDrive/Krishnin/Davidson_sampled_with_emotion_test.csv')

Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

1800565